# 05 - Pipeline Demo ResQPet

Dimostrazione della pipeline completa per la detection di cani randagi.

## Pipeline
```
Input Image
    │
    ▼
┌───────────────────┐
│   YOLO11 Pose     │ → Keypoints (24 punti)
│   (Backbone)      │ → Bounding Box
└───────────────────┘
    │
    ├──────────────────┬──────────────────┬──────────────────┐
    ▼                  ▼                  ▼                  ▼
┌─────────┐      ┌─────────┐      ┌─────────┐      ┌─────────┐
│ Collar  │      │  Skin   │      │  Pose   │      │ Breed   │
│Detector │      │Classifier│     │Classifier│     │Classifier│
└─────────┘      └─────────┘      └─────────┘      └─────────┘
    │                  │                  │                  │
    ▼                  ▼                  ▼                  ▼
P(no_collar)      P(disease)       P(stray_pose)    P(stray|breed)
    │                  │                  │                  │
    └──────────────────┴──────────────────┴──────────────────┘
                              │
                              ▼
                    ┌─────────────────┐
                    │  Fusion Module  │
                    │   (Weighted)    │
                    └─────────────────┘
                              │
                              ▼
                      STRAY INDEX [0,1]
```

## Weights
- Collar: 0.35 (indicatore forte)
- Skin: 0.20 (condizioni sanitarie)
- Pose: 0.25 (comportamento)
- Breed: 0.20 (prior statistico)

In [ ]:
# Installazione dipendenze
%pip install ultralytics torch torchvision timm pillow matplotlib opencv-python -q

In [ ]:
import os
import sys
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
from datetime import datetime
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import timm

from ultralytics import YOLO

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")

In [ ]:
# Configurazione paths (usando notebook_utils per portabilità)
import sys
from pathlib import Path

# Aggiungi la directory notebooks al path per importare notebook_utils
notebooks_dir = Path.cwd()
if notebooks_dir.name != "notebooks":
    notebooks_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
if str(notebooks_dir) not in sys.path:
    sys.path.insert(0, str(notebooks_dir))

from notebook_utils import get_paths, get_device, print_paths

# Ottieni tutti i path in modo relativo
paths = get_paths()
print_paths(paths)

# Estrai path principali
PROJECT_DIR = paths['project_dir']
BASE_DIR = paths['base_dir']
WEIGHTS_DIR = paths['weights_dir']
DATA_DIR = paths['data_dir']

# Device auto-detection
DEVICE, DEVICE_NAME = get_device()
print(f"\n🖥️ Device: {DEVICE} ({DEVICE_NAME})")

## 1. Configurazione Pipeline

In [ ]:
@dataclass
class PipelineConfig:
    """Configurazione pipeline ResQPet"""
    # Modelli
    # IMPORTANTE: usa yolo11n-dog-pose.pt (non yolo11n-pose.pt che rileva persone!)
    backbone_model: str = "yolo11n-dog-pose.pt"  # Dog detection with 24 keypoints
    collar_model: str = "collar_detector.pt"
    skin_model: str = "skin_classifier.pt"
    pose_model: str = "stray_pose_classifier.pt"
    breed_model: str = "breed_classifier.pt"
    
    # Pesi fusion
    weight_collar: float = 0.35
    weight_skin: float = 0.20
    weight_pose: float = 0.25
    weight_breed: float = 0.20
    
    # Soglie
    stray_threshold: float = 0.6
    confidence_threshold: float = 0.5
    
    # Image processing
    img_size: int = 640
    classifier_img_size: int = 224

config = PipelineConfig()
print("Configurazione pipeline:")
for key, value in config.__dict__.items():
    print(f"  {key}: {value}")

In [ ]:
@dataclass
class DogDetection:
    """Risultato detection per un singolo cane"""
    # Bounding box
    bbox: Tuple[int, int, int, int]  # x1, y1, x2, y2
    confidence: float
    
    # Keypoints
    keypoints: Optional[np.ndarray] = None
    
    # Probabilità individuali
    p_no_collar: float = 0.5
    p_disease: float = 0.5
    p_stray_pose: float = 0.5
    p_stray_breed: float = 0.5
    
    # Breed info
    predicted_breed: str = "unknown"
    breed_confidence: float = 0.0
    
    # Stray Index finale
    stray_index: float = 0.5
    is_stray: bool = False
    
    def to_dict(self) -> dict:
        return {
            'bbox': self.bbox,
            'confidence': self.confidence,
            'stray_index': self.stray_index,
            'is_stray': self.is_stray,
            'components': {
                'p_no_collar': self.p_no_collar,
                'p_disease': self.p_disease,
                'p_stray_pose': self.p_stray_pose,
                'p_stray_breed': self.p_stray_breed
            },
            'breed': {
                'predicted': self.predicted_breed,
                'confidence': self.breed_confidence
            }
        }

## 2. Caricamento Modelli

In [ ]:
class StrayPoseMLP(nn.Module):
    """MLP per classificazione pose (copia dal notebook 03)"""
    def __init__(self, input_dim=72, hidden_dims=[128, 64], dropout=0.3):
        """
        Args:
            input_dim: 72 per Dog-Pose (24 keypoints × 3), 51 per COCO (17 keypoints × 3)
        """
        super().__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_dim),  # BatchNorm PRIMA di Dropout
                nn.Dropout(dropout)
            ])
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, 1))
        layers.append(nn.Sigmoid())
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x).squeeze(-1)


class BreedClassifier(nn.Module):
    """EfficientNet per classificazione razze (copia dal notebook 04)"""
    def __init__(self, num_classes, pretrained=False):
        super().__init__()
        self.backbone = timm.create_model('efficientnet_b0', pretrained=pretrained, num_classes=0)
        self.feature_dim = self.backbone.num_features
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.feature_dim, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        self.num_classes = num_classes
    
    def forward(self, x):
        features = self.backbone(x)
        return self.classifier(features)
    
    def predict_proba(self, x):
        return torch.softmax(self.forward(x), dim=-1)

In [ ]:
class ResQPetPipeline:
    """Pipeline completa per detection cani randagi"""
    
    def __init__(self, config: PipelineConfig, weights_dir: Path, device: torch.device):
        self.config = config
        self.weights_dir = weights_dir
        self.device = device
        self.pose_input_dim = 72  # Default: 24 keypoints × 3 (Dog-Pose)
        
        # Carica modelli
        self._load_models()
        
        # Trasformazioni per classificatori
        self.classifier_transform = transforms.Compose([
            transforms.Resize((config.classifier_img_size, config.classifier_img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        # Carica breed priors
        priors_file = weights_dir.parent.parent / 'data' / 'breed_priors.json'
        if priors_file.exists():
            with open(priors_file) as f:
                self.breed_priors = json.load(f)
        else:
            self.breed_priors = {'unknown': 0.5}
    
    def _load_models(self):
        """Carica tutti i modelli"""
        print("Caricamento modelli...")
        
        # 1. Backbone (YOLO11 Pose) - scarica se non esiste
        backbone_path = self.weights_dir / self.config.backbone_model
        if backbone_path.exists():
            self.backbone = YOLO(str(backbone_path))
            self.use_pose_model = True
            print(f"  ✓ Backbone: {backbone_path.name}")
        else:
            # Scarica yolo11n-pose automaticamente da ultralytics
            print(f"  ⏳ Scaricamento yolo11n-pose.pt...")
            try:
                self.backbone = YOLO('yolo11n-pose.pt')  # Ultralytics scarica automaticamente
                self.use_pose_model = True
                print(f"  ✓ Backbone: yolo11n-pose.pt (scaricato)")
            except Exception as e:
                print(f"  ⚠ Impossibile scaricare yolo11n-pose: {e}")
                print(f"  ⚠ Usando yolov8n.pt come fallback")
                self.backbone = YOLO('yolov8n.pt')
                self.use_pose_model = False
        
        # 2. Collar Detector
        collar_path = self.weights_dir / self.config.collar_model
        if collar_path.exists():
            self.collar_detector = YOLO(str(collar_path))
            print(f"  ✓ Collar Detector: {collar_path.name}")
        else:
            self.collar_detector = None
            print(f"  ⚠ Collar Detector: usando euristica")
        
        # 3. Skin Classifier (weights_only=False per PyTorch 2.6+)
        skin_path = self.weights_dir / self.config.skin_model
        if skin_path.exists():
            checkpoint = torch.load(skin_path, map_location=self.device, weights_only=False)
            self.skin_classifier = timm.create_model('resnet50', pretrained=False, num_classes=checkpoint.get('num_classes', 6))
            self.skin_classifier.load_state_dict(checkpoint['model_state_dict'])
            self.skin_classifier.to(self.device)
            self.skin_classifier.eval()
            print(f"  ✓ Skin Classifier: {skin_path.name}")
        else:
            self.skin_classifier = None
            print(f"  ⚠ Skin Classifier: usando euristica")
        
        # 4. Pose Classifier (weights_only=False per PyTorch 2.6+)
        pose_path = self.weights_dir / self.config.pose_model
        if pose_path.exists():
            checkpoint = torch.load(pose_path, map_location=self.device, weights_only=False)
            # IMPORTANTE: leggi input_dim dal checkpoint per compatibilità
            self.pose_input_dim = checkpoint.get('input_dim', 72)  # 72 per Dog-Pose, 51 per COCO
            hidden_dims = checkpoint.get('hidden_dims', [128, 64])
            self.pose_classifier = StrayPoseMLP(input_dim=self.pose_input_dim, hidden_dims=hidden_dims)
            self.pose_classifier.load_state_dict(checkpoint['model_state_dict'])
            self.pose_classifier.to(self.device)
            self.pose_classifier.eval()
            print(f"  ✓ Pose Classifier: {pose_path.name} (input_dim={self.pose_input_dim})")
        else:
            self.pose_classifier = None
            print(f"  ⚠ Pose Classifier: usando euristica")
        
        # 5. Breed Classifier (weights_only=False per PyTorch 2.6+)
        breed_path = self.weights_dir / self.config.breed_model
        if breed_path.exists():
            checkpoint = torch.load(breed_path, map_location=self.device, weights_only=False)
            num_classes = len(checkpoint.get('categories', ['unknown']))
            self.breed_classifier = BreedClassifier(num_classes=num_classes)
            self.breed_classifier.load_state_dict(checkpoint['model_state_dict'])
            self.breed_classifier.to(self.device)
            self.breed_classifier.eval()
            self.breed_categories = checkpoint.get('categories', ['unknown'])
            print(f"  ✓ Breed Classifier: {breed_path.name}")
        else:
            self.breed_classifier = None
            self.breed_categories = ['unknown']
            print(f"  ⚠ Breed Classifier: usando euristica")
        
        print("Modelli caricati!")
    
    def preprocess_keypoints(self, keypoints: np.ndarray) -> np.ndarray:
        """
        Preprocessa keypoints per il classificatore pose.
        
        Gestisce dinamicamente:
        - 24 keypoints (Dog-Pose) → 72 features
        - 17 keypoints (COCO Human) → 51 features
        
        L'output deve matchare self.pose_input_dim del modello caricato.
        """
        num_keypoints = len(keypoints)
        target_dim = self.pose_input_dim  # 72 o 51 dal modello
        
        # Filtra keypoints con bassa confidence
        valid_kpts = keypoints[keypoints[:, 2] > 0.3]
        if len(valid_kpts) < 3:
            return np.zeros(target_dim)
        
        # Normalizza rispetto al centro e scala
        center_x = valid_kpts[:, 0].mean()
        center_y = valid_kpts[:, 1].mean()
        scale = max(valid_kpts[:, 0].max() - valid_kpts[:, 0].min(),
                   valid_kpts[:, 1].max() - valid_kpts[:, 1].min()) + 1e-6
        
        normalized = keypoints.copy()
        normalized[:, 0] = (keypoints[:, 0] - center_x) / scale
        normalized[:, 1] = (keypoints[:, 1] - center_y) / scale
        # La confidence (colonna 2) rimane invariata
        
        # Flatten e adatta alla dimensione target
        features = normalized.flatten()
        
        if len(features) < target_dim:
            # Pad con zeri se ci sono meno keypoints del previsto
            features = np.pad(features, (0, target_dim - len(features)))
        elif len(features) > target_dim:
            # Trunca se ci sono più keypoints del previsto
            features = features[:target_dim]
        
        return features
    
    def _predict_collar(self, image: Image.Image) -> float:
        """Predice P(no_collar)"""
        if self.collar_detector is None:
            return 0.5
        
        results = self.collar_detector(image, verbose=False)
        
        for result in results:
            for box in result.boxes:
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                if cls == 0:  # Con guinzaglio
                    return 1.0 - conf
                elif cls == 1:  # Senza guinzaglio
                    return conf
        
        return 0.7  # Default: probabilmente senza
    
    def _predict_skin(self, image: Image.Image) -> float:
        """Predice P(disease)"""
        if self.skin_classifier is None:
            return 0.3
        
        img_tensor = self.classifier_transform(image).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            output = self.skin_classifier(img_tensor)
            probs = torch.softmax(output, dim=-1)
            # Somma probabilità di tutte le classi di malattia (non Healthy)
            # Assumendo che classe 0 sia Healthy
            p_disease = 1.0 - probs[0, 0].item()
            return p_disease
    
    def _predict_pose(self, keypoints: np.ndarray) -> float:
        """Predice P(stray_pose)"""
        if self.pose_classifier is None or keypoints is None:
            return 0.5
        
        features = self.preprocess_keypoints(keypoints)
        features_tensor = torch.FloatTensor(features).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            output = self.pose_classifier(features_tensor)
            return output.item()
    
    def _predict_breed(self, image: Image.Image) -> Tuple[str, float, float]:
        """Predice razza e ritorna P(stray|breed)"""
        if self.breed_classifier is None:
            return 'unknown', 0.0, 0.5
        
        img_tensor = self.classifier_transform(image).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            probs = self.breed_classifier.predict_proba(img_tensor)
            top_prob, top_idx = probs.max(dim=1)
            
            breed = self.breed_categories[top_idx.item()]
            confidence = top_prob.item()
            stray_prior = self.breed_priors.get(breed, 0.5)
            
            return breed, confidence, stray_prior
    
    def compute_stray_index(self, p_collar: float, p_skin: float, 
                           p_pose: float, p_breed: float) -> float:
        """Calcola Stray Index pesato"""
        weights = [
            self.config.weight_collar,
            self.config.weight_skin,
            self.config.weight_pose,
            self.config.weight_breed
        ]
        probabilities = [p_collar, p_skin, p_pose, p_breed]
        stray_index = sum(w * p for w, p in zip(weights, probabilities))
        return np.clip(stray_index, 0, 1)
    
    def process_image(self, image_path: str) -> List[DogDetection]:
        """Processa un'immagine e ritorna le detection"""
        image = Image.open(image_path).convert('RGB')
        
        # Detection con backbone
        results = self.backbone(image, verbose=False)
        
        detections = []
        
        for result in results:
            for i, box in enumerate(result.boxes):
                cls = int(box.cls[0])
                
                # Per modelli pose: classe 0 = persona, per COCO: classe 16 = cane
                # YOLO11-pose è trainato su persone, quindi accettiamo tutte le detection
                # e usiamo il collar_detector per filtrare
                if not self.use_pose_model:
                    # Per yolov8n generico, filtra solo cani (classe 16 COCO)
                    if cls != 16:
                        continue
                
                conf = float(box.conf[0])
                if conf < self.config.confidence_threshold:
                    continue
                
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                
                # Keypoints (se disponibili)
                keypoints = None
                if hasattr(result, 'keypoints') and result.keypoints is not None:
                    if i < len(result.keypoints):
                        keypoints = result.keypoints[i].data[0].cpu().numpy()
                
                # Crop immagine per classificatori
                crop = image.crop((x1, y1, x2, y2))
                
                # Predizioni individuali
                p_collar = self._predict_collar(crop)
                p_skin = self._predict_skin(crop)
                p_pose = self._predict_pose(keypoints)
                breed, breed_conf, p_breed = self._predict_breed(crop)
                
                # Stray Index
                stray_index = self.compute_stray_index(p_collar, p_skin, p_pose, p_breed)
                is_stray = stray_index >= self.config.stray_threshold
                
                detection = DogDetection(
                    bbox=(x1, y1, x2, y2),
                    confidence=conf,
                    keypoints=keypoints,
                    p_no_collar=p_collar,
                    p_disease=p_skin,
                    p_stray_pose=p_pose,
                    p_stray_breed=p_breed,
                    predicted_breed=breed,
                    breed_confidence=breed_conf,
                    stray_index=stray_index,
                    is_stray=is_stray
                )
                detections.append(detection)
        
        return detections

In [ ]:
# Inizializza pipeline
pipeline = ResQPetPipeline(config, WEIGHTS_DIR, DEVICE)

## 3. Demo su Immagini

In [ ]:
def visualize_detection(image_path: str, detections: List[DogDetection], 
                        save_path: Optional[str] = None):
    """Visualizza le detection su un'immagine"""
    image = Image.open(image_path)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Immagine con bounding box
    axes[0].imshow(image)
    
    for det in detections:
        x1, y1, x2, y2 = det.bbox
        
        # Colore basato su stray_index
        if det.is_stray:
            color = 'red'
            label = f"STRAY ({det.stray_index:.0%})"
        else:
            color = 'green'
            label = f"OWNED ({det.stray_index:.0%})"
        
        # Bounding box
        rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                                  linewidth=3, edgecolor=color, facecolor='none')
        axes[0].add_patch(rect)
        
        # Label
        axes[0].text(x1, y1-10, label, color=color, fontsize=12, 
                    fontweight='bold', backgroundcolor='white')
        
        # Keypoints
        if det.keypoints is not None:
            for kpt in det.keypoints:
                if kpt[2] > 0.3:  # Confidence > 0.3
                    axes[0].scatter(kpt[0], kpt[1], c='yellow', s=30, zorder=5)
    
    axes[0].set_title('Detection Results')
    axes[0].axis('off')
    
    # Breakdown probabilità
    if detections:
        det = detections[0]  # Prima detection
        
        components = ['No Collar', 'Disease', 'Stray Pose', f'Breed ({det.predicted_breed})']
        probs = [det.p_no_collar, det.p_disease, det.p_stray_pose, det.p_stray_breed]
        weights = [config.weight_collar, config.weight_skin, 
                  config.weight_pose, config.weight_breed]
        
        # Bar chart
        y_pos = np.arange(len(components))
        colors = plt.cm.RdYlGn_r(probs)
        
        bars = axes[1].barh(y_pos, probs, color=colors)
        axes[1].set_yticks(y_pos)
        axes[1].set_yticklabels([f"{c}\n(w={w})" for c, w in zip(components, weights)])
        axes[1].set_xlim(0, 1)
        axes[1].set_xlabel('Probability')
        axes[1].set_title(f'Stray Index Breakdown\nFinal: {det.stray_index:.2%}')
        
        # Aggiungi valori sulle barre
        for bar, prob in zip(bars, probs):
            axes[1].text(prob + 0.02, bar.get_y() + bar.get_height()/2,
                        f'{prob:.2f}', va='center')
        
        # Linea threshold
        axes[1].axvline(x=config.stray_threshold, color='red', 
                       linestyle='--', label=f'Threshold ({config.stray_threshold})')
    else:
        axes[1].text(0.5, 0.5, 'No dogs detected', ha='center', va='center')
        axes[1].axis('off')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
    
    plt.show()
    
    return fig

In [ ]:
# Trova immagini di test usando i path relativi
test_dirs = [
    paths['fyp_dataset'] / "images" / "testing",      # FYP stray dogs
    paths['collar_dataset'] / "split_dataset" / "test" / "images",  # Collar dataset
    paths['skin_dataset'] / "test" / "Healthy",       # Skin - cani sani
    paths['skin_dataset'] / "test" / "Fungal_infections",  # Skin - malati
    paths['stanford_dogs'] / "Images" / "n02085620-Chihuahua",  # Stanford Dogs
]

test_images = []
for test_dir in test_dirs:
    if test_dir.exists():
        images = list(test_dir.glob('*.jpg')) + list(test_dir.glob('*.png'))
        test_images.extend(images[:2])  # Prendi 2 immagini per directory
        print(f"✓ {test_dir.name}: {len(images)} immagini trovate")
    else:
        print(f"✗ {test_dir} non trovato")

print(f"\nTotale: {len(test_images)} immagini di test")
for img in test_images[:10]:
    print(f"  - {img.parent.name}/{img.name}")

In [ ]:
# Processa immagini di test
if test_images:
    for img_path in test_images[:4]:
        print(f"\n{'='*50}")
        print(f"Processing: {img_path.name}")
        print(f"{'='*50}")
        
        # Processa
        detections = pipeline.process_image(str(img_path))
        
        print(f"\nRilevati {len(detections)} cani:")
        for i, det in enumerate(detections):
            print(f"\nCane #{i+1}:")
            print(f"  Confidence: {det.confidence:.2%}")
            print(f"  Stray Index: {det.stray_index:.2%}")
            print(f"  Is Stray: {'SÌ ⚠️' if det.is_stray else 'NO ✓'}")
            print(f"  Components:")
            print(f"    - P(no_collar): {det.p_no_collar:.2f}")
            print(f"    - P(disease): {det.p_disease:.2f}")
            print(f"    - P(stray_pose): {det.p_stray_pose:.2f}")
            print(f"    - P(stray|{det.predicted_breed}): {det.p_stray_breed:.2f}")
        
        # Visualizza - salva nella directory runs
        output_dir = paths['runs_dir'] / 'demo'
        output_dir.mkdir(parents=True, exist_ok=True)
        visualize_detection(
            str(img_path), 
            detections,
            str(output_dir / f'demo_{img_path.stem}.png')
        )
else:
    print("Nessuna immagine di test trovata")

## 4. Analisi Sensibilità Pesi

In [ ]:
def analyze_weight_sensitivity(detection: DogDetection):
    """
    Analizza come variano i risultati al variare dei pesi
    """
    probs = [detection.p_no_collar, detection.p_disease, 
             detection.p_stray_pose, detection.p_stray_breed]
    labels = ['Collar', 'Skin', 'Pose', 'Breed']
    
    # Varia ciascun peso
    weight_range = np.linspace(0.1, 0.5, 20)
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    base_weights = [0.35, 0.20, 0.25, 0.20]
    
    for i, (ax, label) in enumerate(zip(axes, labels)):
        stray_indices = []
        
        for w in weight_range:
            # Modifica peso i e rinormalizza
            weights = base_weights.copy()
            weights[i] = w
            total = sum(weights)
            weights = [ww/total for ww in weights]
            
            stray_idx = sum(ww * p for ww, p in zip(weights, probs))
            stray_indices.append(stray_idx)
        
        ax.plot(weight_range, stray_indices, 'b-', linewidth=2)
        ax.axhline(y=0.6, color='r', linestyle='--', label='Threshold')
        ax.axvline(x=base_weights[i], color='g', linestyle=':', 
                  label=f'Current ({base_weights[i]})')
        ax.fill_between(weight_range, 0.6, stray_indices, 
                       where=[s >= 0.6 for s in stray_indices],
                       alpha=0.3, color='red')
        ax.set_xlabel(f'{label} Weight')
        ax.set_ylabel('Stray Index')
        ax.set_title(f'Sensitivity to {label} Weight')
        ax.set_ylim(0, 1)
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.suptitle(f'Weight Sensitivity Analysis\nP=[{probs[0]:.2f}, {probs[1]:.2f}, {probs[2]:.2f}, {probs[3]:.2f}]')
    plt.tight_layout()
    
    # Salva nella directory runs
    output_dir = paths['runs_dir'] / 'demo'
    output_dir.mkdir(parents=True, exist_ok=True)
    plt.savefig(output_dir / 'weight_sensitivity.png', dpi=150)
    plt.show()

# Crea detection di esempio
example_detection = DogDetection(
    bbox=(0, 0, 100, 100),
    confidence=0.9,
    p_no_collar=0.8,
    p_disease=0.3,
    p_stray_pose=0.6,
    p_stray_breed=0.7,
    stray_index=0.65,
    is_stray=True
)

analyze_weight_sensitivity(example_detection)

## 5. Simulazione CCTV

In [ ]:
def simulate_cctv_frame(pipeline: ResQPetPipeline, image_path: str) -> dict:
    """
    Simula l'elaborazione di un frame CCTV
    Ritorna dati nel formato atteso dal frontend
    """
    import time
    
    start_time = time.time()
    
    # Processa
    detections = pipeline.process_image(image_path)
    
    processing_time = time.time() - start_time
    
    # Formato per frontend
    response = {
        'timestamp': datetime.now().isoformat(),
        'camera_id': 'CAM_001',
        'processing_time_ms': processing_time * 1000,
        'dogs_detected': len(detections),
        'alerts': [],
        'detections': []
    }
    
    for det in detections:
        response['detections'].append(det.to_dict())
        
        if det.is_stray:
            alert = {
                'type': 'STRAY_DETECTED',
                'severity': 'high' if det.stray_index > 0.8 else 'medium',
                'stray_index': det.stray_index,
                'bbox': det.bbox,
                'message': f"Possibile cane randagio rilevato (Index: {det.stray_index:.0%})"
            }
            response['alerts'].append(alert)
    
    return response

In [ ]:
# Simula elaborazione CCTV
if test_images:
    print("="*50)
    print("SIMULAZIONE CCTV")
    print("="*50)
    
    for img_path in test_images[:3]:
        result = simulate_cctv_frame(pipeline, str(img_path))
        
        print(f"\n--- Frame: {img_path.name} ---")
        print(f"Camera: {result['camera_id']}")
        print(f"Processing: {result['processing_time_ms']:.1f}ms")
        print(f"Dogs detected: {result['dogs_detected']}")
        
        if result['alerts']:
            print("\n⚠️ ALERTS:")
            for alert in result['alerts']:
                print(f"  [{alert['severity'].upper()}] {alert['message']}")
        else:
            print("\n✓ No alerts")
        
        print(f"\nJSON Response:")
        print(json.dumps(result, indent=2, default=str))

## 6. Riepilogo e Prossimi Passi

In [ ]:
print("\n" + "="*60)
print("RIEPILOGO PIPELINE ResQPet")
print("="*60)

print("\n📊 ARCHITETTURA:")
print("  ┌─────────────────────────────────────────────┐")
print("  │           YOLO11 Dog Pose (Backbone)        │")
print("  │        Detection + Keypoint Extraction      │")
print("  └─────────────────────────────────────────────┘")
print("                        │")
print("        ┌───────────────┼───────────────┐")
print("        ▼               ▼               ▼")
print("  ┌──────────┐   ┌──────────┐   ┌──────────┐")
print("  │ Collar   │   │  Skin    │   │  Pose    │")
print("  │ (YOLO)   │   │(ResNet50)│   │  (MLP)   │")
print("  └──────────┘   └──────────┘   └──────────┘")
print("        │               │               │")
print("        └───────────────┼───────────────┘")
print("                        ▼")
print("              ┌─────────────────┐")
print("              │ Fusion (Weighted)│")
print("              └─────────────────┘")
print("                        │")
print("                        ▼")
print("                  STRAY INDEX")

print("\n⚖️ PESI FUSION:")
print(f"  • Collar:  {config.weight_collar:.0%} (indicatore principale)")
print(f"  • Pose:    {config.weight_pose:.0%} (comportamento)")
print(f"  • Skin:    {config.weight_skin:.0%} (condizioni)")
print(f"  • Breed:   {config.weight_breed:.0%} (prior statistico)")

print("\n📁 MODELLI DISPONIBILI:")
models = [
    ("Backbone (YOLO11 Pose)", config.backbone_model),
    ("Collar Detector", config.collar_model),
    ("Skin Classifier", config.skin_model),
    ("Pose Classifier", config.pose_model),
    ("Breed Classifier", config.breed_model)
]

for name, filename in models:
    path = WEIGHTS_DIR / filename
    status = "✓" if path.exists() else "⚠️ (usando euristica)"
    print(f"  {status} {name}: {filename}")

print("\n🔧 PROSSIMI PASSI:")
print("  1. Eseguire i notebook 00-04 per allenare i modelli")
print("  2. Avviare il backend: cd backend && python run.py")
print("  3. Avviare il frontend: cd frontend && npm run dev")
print("  4. Accedere a http://localhost:5173 per la GUI CCTV")
